In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, regularizers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import kerastuner as kt
from sklearn.model_selection import train_test_split

C:\Users\ldani\AppData\Local\Temp\ipykernel_18172\1306372946.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
# Paths to the dataset
lung_path = r'C:\Users\ldani\Documents\Patronus\Challenges\heart-and-lung-segmentations-for-mimic-cxrmimic-cxr-jpg-and-montgomery-county-tb-databases-1.0.0\mimic_masks\lungs'
heart_path = r'C:\Users\ldani\Documents\Patronus\Challenges\heart-and-lung-segmentations-for-mimic-cxrmimic-cxr-jpg-and-montgomery-county-tb-databases-1.0.0\mimic_masks\heart'

# Load and prepare the dataset
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = tf.keras.preprocessing.image.load_img(os.path.join(folder, filename), color_mode='grayscale')
        if img is not None:
            img = tf.keras.preprocessing.image.img_to_array(img)
            img = tf.image.resize(img, (128, 128))  # Adjust the size as needed
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

lung_images, lung_labels = load_images_from_folder(lung_path, 0)
heart_images, heart_labels = load_images_from_folder(heart_path, 1)


In [4]:

# Combine and split the data
images = np.concatenate((lung_images, heart_images), axis=0)
labels = np.concatenate((lung_labels, heart_labels), axis=0)

# Normalize images
images = images / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [5]:

# Definir la función para construir el modelo con Keras Tuner
def constructor_modelos(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
                                     kernel_size=(3,3), activation='relu', input_shape=(128, 128, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(filters=hp.Int('filters_2', min_value=64, max_value=256, step=64),
                                     kernel_size=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(filters=hp.Int('filters_3', min_value=128, max_value=512, step=128),
                                     kernel_size=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation="relu", kernel_regularizer=regularizers.l2(1e-5)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=hp_units, activation="relu", kernel_regularizer=regularizers.l2(1e-5)))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(1, activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [6]:

# Configurar el tuner
tuner = kt.Hyperband(
    constructor_modelos,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)


c:\Users\ldani\Documents\Patronus\Challenges\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:

# Buscar los mejores hiperparámetros
tuner.search(x_train, y_train, epochs=20, validation_split=0.2)

# Recuperar los mejores hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Construir el modelo con los mejores hiperparámetros
hypermodel = tuner.hypermodel.build(best_hps)

# Definir los callbacks para el entrenamiento
callback_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
callbacks = [callback_early, callback_checkpoint]


Trial 30 Complete [00h 07m 33s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 45m 09s


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5

In [8]:

# Entrenar el modelo con los callbacks
history_hypermodel = hypermodel.fit(
    x_train, y_train,
    epochs=20,
    validation_split=0.2,
    callbacks=callbacks
)

# Evaluar el modelo en el conjunto de prueba
results = hypermodel.evaluate(x_test, y_test)
print(f'Test accuracy: {results[1]}')

# Gráficos de pérdida y precisión
history_dict = history_hypermodel.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


NameError: name 'callbacks' is not defined